# Boundary conditions

    For our first model, we have chosen the following conditions to simplify as much as possible :
        - The buildings exchanges with the outdoor, the outdoor temperature is homogenous
        - The radiations of the sun are diffuse, we have the same flux everywhere on the walls
        - We consider that the heat exchanges with the ground are negligible, we suppose that the floor is perfectly insulated
        - We consider that the roof top has the same thermical properties as the walls
        - We consider that the indoor temperature is homogenous
        - We neglect long wave radiation
        
    The final flux that we have for this model are : 
        - Sun's radiaitons that are homogenous all around the building, including the roof 
        - Conductive and convective exchanges with the outdoor

In [38]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import dm4bem

ModuleNotFoundError: No module named 'dm4bem'

The wall is composed of two materials: concrete and insulation. The wall 1 in normally insolated and wall 2 is supposed to be perfectly insolated. The windows are simple glazed glass windows.  The door is in PVC. The thermal properties of the materials of the wall are:

Let's consider the surface area of the walls 6 x 3 m² & 10 x 3 m² and the volume of the indoor air is 6 x 10 x 3 m³.

In [57]:
V_air = 6 * 10 * 3   # m³, indoor air volume

In [ ]:
air = {'Density': 1.2,                      # kg/m³
       'Specific heat': 1000}               # J/(kg·K)
# pd.DataFrame.from_dict(air, orient='index', columns=['air'])
pd.DataFrame(air, index=['Air'])

,Density,Specific heat
Air,1.2,1000


In [ ]:
concrete = {'Conductivity': 1.400,
            'Density': 2300.0,
            'Specific heat': 880,
            'Width': 0.2,
            'Surface': 5*3-1*3+10*3-1.2*3+5*3-1*3+10*3}

insulation = {'Conductivity': 0.040,
              'Density': 16.0,
              'Specific heat': 1210,
              'Width': 0.1,
              'Surface': 5*3-1*3+10*3-1.2*3+5*3-1*3+10*3}



glass = {'Conductivity': 1.4,
         'Density': 2500,
         'Specific heat': 1210,
         'Width': 0.04,
         'Surface': 2*3}
         
pvc = {'Conductivity': 0.2,
         'Density': 1.4,
         'Specific heat': 840,
         'Width': 0.05,
         'Surface': 1.2*3}
         

wall = pd.DataFrame.from_dict({'Layer_out': insulation,
                               'Layer_in': concrete,
                               'Glass': glass,
                               'Pvc': pvc },
                              orient='index')
wall

SyntaxError: invalid syntax (3026862523.py, line 5)

In [ ]:
air = {'Density': 1.2,
       'Specific heat': 1000}

pd.DataFrame(air, index=['Air'])

,Density,Specific heat
Air,1.2,1000


In [ ]:
# convection coefficients, W/(m²·K)
h = pd.DataFrame([{'in': 4+4., 'out': 10.}], index=['h'])
h

,in,out
h,8.0,10.0


# Let us start to draw the thermal circuit
## Thermal circuit

Heat transfert is:
- through the walls (concrete and insulation), 
- through the double glazed glass windows,

The HVAC system is modelled as a proportional controller. There is long wave radiative exchange between the wall and the glass window. The sources are:
- temperature sources:
    - outdoor atmospheric air;
    - indoor air temperature setpoint;
- flow rate sources:
    - solar radiation on the outdoor and the indoor walls;
    - auxiliary heat gains in the thermal zone.


### Let us consider the thermal conductances for conduction, convection, long-wave radiation, and advection.

#### Thermal conductance
$$G_{cd} = \frac{\lambda}{w}S$$


In [ ]:
# conduction
G_cd = wall['Conductivity'] / wall['Width'] * wall['Surface']
pd.DataFrame(G_cd, columns={'Conductance'})

KeyError: 'Surface'

#### Convection
For the inside convection coefficient, we add 4 (by convention) in order to consider easily the radiating factor. 

In [39]:
# convection
Gw = h * wall['Surface'][0]     # wall
Gg = h * wall['Surface'][2]     # glass
Gd= h*wall['Surface'][3]        # door pvc


KeyError: 'Surface'

#### Advection

In [42]:
# ventilation flow rate
Va = V_air                  # m³, volume of air
ACH = 1                     # air changes per hour
Va_dot = ACH / 3600 * Va    # m³/s, air infiltration
# ventilation & advection
Gv = air['Density'] * air['Specific heat'] * Va_dot


> Table 1. Typical values for the ventilation rates (in air changes per hour, ACH) as a function of the position of windows (H. Recknagel, E. Spenger, E_R Schramek (2013), Table 1.12.1-4)

| Position of windows                     | Ventilation rate, ACH [h⁻ⁱ] |
| --------------------------------------- | ---------------------- |
| Window closed, doors closed             | 0 to 0.5 |
| Tilted window, venetian blind closed    | 0.3 to 1.5 |
| Tilted window, whitout venetian blind   | 0.8 to 4.0 |
| Window half opened                      | 5 to 10 |
| Window fully open                       | 9 to 15 |
| Window and French window fully open (cross ventilation) | about 40 |


In [43]:
# glass: convection outdoor & conduction
Ggs = float(1 / (1 / Gg['out'] + 1 / (2 * G_cd['Glass'])))

NameError: name 'Gg' is not defined

#### Resistances

We have

$$R_{cd} = \frac{w}{\lambda S}$$

where:

- $w$ is the [width](https://en.m.wikipedia.org/wiki/Length) of the material, m;
- $\lambda$ - [thermal conductvity](https://en.m.wikipedia.org/wiki/Thermal_conductivity) of the material, W/(m·K);
- $S$ - [surface area](https://en.m.wikipedia.org/wiki/Surface_area) of the wall, m².

In [ ]:
# conduction
R_cd_1_3 = wall['Width'] / (wall['Conductivity'] * S_wall_1_3)  # K/W
R_cd_2_4 = wall['Width'] / (wall['Conductivity'] * S_wall_2_4)

#### Thermal Capacities
The [thermal capacities](https://en.m.wikipedia.org/wiki/Heat_capacity) of the wall are of the form:

$$C_w= m_w c_w= \rho_w c_w w_w S_w$$

where:
- $m_w = \rho_w w_w S_w$ is the [mass](https://en.m.wikipedia.org/wiki/Mass) of the material of the wall, kg;
- $c_w$ - [specific heat capacity](https://en.m.wikipedia.org/wiki/Specific_heat_capacity) of the material, J/(kg⋅K);
- $\rho_w$ - [density](https://en.m.wikipedia.org/wiki/Density) of the material, kg/m³;
- $w_w$ - width of the wall, m;
- $S_w$ - surface area of the wall, m²

In [44]:
C = wall['Density'] * wall['Specific heat'] * wall['Surface'] * wall['Width']

KeyError: 'Surface'

The thermal capacity of the air is:

$$C_a = m_a c_a = \rho_a c_a V_a$$

where:
- $m_a = \rho_a V_a$ is the mass of the air, kg;
- $\rho_w$ - [density](https://en.m.wikipedia.org/wiki/Density) of air, kg/m³;
- $c_a$ - specific heat capacity of the air, J/(kg⋅K);
- $V_a$ - volume of the air in the thermal zone, m³.

In [45]:
C['Air'] = air['Density'] * air['Specific heat'] * V_air
pd.DataFrame(C, columns={'Capacity'})

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

## Differential algebraic equations (DAE)

In [46]:
# number of temperature nodes and flow branches
no_θ = 18 #nodes
no_q = 26 #flow

### Conductance matrix $G$

In [58]:
he=h["out"] #h ext
hi=h["in"] #h interieur
Sint2=5.4*5.6 #surface au sol piece 2
Sm1=3*5*3-1*3-1.2*3 #surface mur 1
Sm12=4.4*3 #surface mur mileu
Sp=1.2*3 # surface portes
Sf=1*3 #surface fenetres
eb=0.2 #epaisseur beton
ei=0.1 #epaisseur isolant
ef=0.01 #epaisseur fenetre
ep=0.05 # epaisseur porte
em12=0.1 #epaisseur mur mileu
lda_b =1.4 #conductivité thermique du beton
lda_i = 0.04#conductivité du beton
lda_p = 0.2#conductivité pvc
lda_f = 1.4#conductivité vitre
dens_b = 2300#densité beton
dens_i = 16#densité isolant
dens_f = 2500#densité fenêtre
dens_p = 1410#densité porte
dens_air=1.2 #densite air
cb=880 #capacité thermique beton
ci=1210
cf=1210
cp=840
cair=1000

R0=1/(he*Sm1) 
R1=R2=eb/(2*lda_b*Sm1) #accumulation de chaleur beton
R3=R4=ei/(2*lda_i*Sm1) #accumulation de chaleur isolant
R5=1/(he*Sf)+ef/(2*lda_f*Sf) #convection fenetre -ext + moitié conduction
R6=1/(he*Sp) #convectiopn ext-porte
R7=R8=ep/(2*lda_p*Sp) #accumulation
R9=1/(hi*Sm1) #convection mur -int
R10=1/(hi*Sf)+ef/(2*lda_f*Sf) #convection fenetre-int + moitié conduction
R11=1/(hi*Sp) #convection porte int
R12=1e9
R13=1e9 # a changer
R14=1/(hi*Sp) #convection interieur porte
R15=1/(hi*Sm12) 
R16=R17=em12/(2*lda_p*Sp)
R18=1/(hi*Sp)
R19=R20=em12/(2*lda_b*Sm12)
R21=1/(hi*Sm12)
R22=R23=1e9
R24=1/(he*Sf)+ef/(2*lda_f*Sf)
R25=1/(hi*Sf)+ef/(2*lda_f*Sf)


SyntaxError: invalid syntax (2417616636.py, line 12)

### Capacity matrix $C$

In [47]:
C[1,0] = dens_b*eb*Sm1*cb
C[3,0] = dens_i*ei*Sm1*ci
C[5,0] = dens_f*ef*Sf*cf
C[7,0] = dens_p*ep*Sp*cp
C[9,0] = dens_air*Sint2*3*cair
C[11,0] = dens_p*Sp*ep*cp
C[14,0] = dens_b*Sm12*em12*cb
C[16,0] = dens_air*Sint2*3*cair
C[17,0] = Sf*ef*cf*dens_f


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

### Arc-node incidence matrix $A$

In [56]:
# Arc-node incidence matrix
A = np.zeros((no_q, no_θ ))
for i in range (10):
    A[i][i]=1
A[4][3]=-1
A[9][4]=-1
A[1][0]=-1
A[2][1]=-1
A[3][2]=-1
A[7][6]=-1
A[8][7]=-1
A[11][8]=-1
A[25][17]=-1
A[24][17]=1
A[23][16]=1
A[21][16]=1
A[20][15]=1
A[19][14]=1
A[18][16]=1
A[17][12]=1
A[16][11]=1
A[15][13]=-1
A[14][10]=-1
A[22][16]=1
A[21][15]=-1
A[20][14]=-1
A[19][13]=-1
A[18][12]=-1
A[17][11]=-1
A[16][10]=-1
A[15][9]=1
A[14][9]=1
A[13][9]=1
A[11][8]=-1
A[10][5]=-1
A[9][4]=-1
A[12][9]=1
A[11][9]=1
A[10][9]=1
pd.DataFrame(A)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,-1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### F

In [ ]:
f = np.zeros((18,1))

f[0,0] = 1
f[4,0] = 1
f[5,0] = 1
f[6,0] = 1
f[8,0] = 1
f[9,0] = 1
f[10,0] = 1
f[12,0] = 1
f[13,0] = 1
f[15,0] = 1
f[17,0] = 1
f[16,0] = 1

b=np.zeros((26,1))

b[0,0]=1
b[5,0]=1
b[6,0]=1
b[12,0]=1
b[13,0]=1
b[22,0]=1
b[23,0]=1
b[24,0]=1